In [1]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import tagui as t
import urllib
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [3]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows

In [4]:
site = getSiteByName("New York Times")
site_url = site[0][2]
site_id = site[0][0]

In [5]:
t.init(visual_automation = True, chrome_browser = True)
t.url(site_url)

True

In [6]:
number = t.count('(//li[contains(@class, "css-1iski2w")]/a)')
df = pd.DataFrame(index=range(0,number), columns = ['Sno', 'Title', 'URL', 'Summary','Img_URL'])

for n in range(1, number+1):
    title=t.read('//li[contains(@class, "css-1iski2w")][{}]/a/div'.format(n))
    URL=t.read('//li[contains(@class, "css-1iski2w")][{}]//@href'.format(n))
    Img_link=t.read('//li[contains(@class, "css-1iski2w")][{}]//img/@src'.format(n))
    summaries = SummarizeUrl(URL)

    urllib.urlretrieve(Img_link, "images/1_{}.jpg".format(n))
    df.iloc[n-1, 0] = n
    df.iloc[n-1, 1] = title
    df.iloc[n-1, 2] = URL
    df.iloc[n-1, 3] = summaries
    df.iloc[n-1, 4] = Img_link

df['Summary'].replace('None', np.nan, inplace=True)
df.dropna(subset=['Summary'], inplace=True, how='any')
df= df.reset_index(drop=True)
df['Sno'] = df.index

c:\users\superhell\.conda\envs\ca1\lib\site-packages\BeautifulSoup.py:114: UserWarning: You are using a very old release of Beautiful Soup, last updated in 2011. If you installed the 'beautifulsoup' package through pip, you should know the 'beautifulsoup' package name is about to be reclaimed by a more recent version of Beautiful Soup which is incompatible with this version.

This will happen at some point after January 1, 2021.

If you just started this project, this is easy to fix. Install the 'beautifulsoup4' package instead of 'beautifulsoup' and start using Beautiful Soup 4.

If this is an existing project that depends on Beautiful Soup 3, the project maintainer (potentially you) needs to start the process of migrating to Beautiful Soup 4. This should be a relatively easy part of the Python 3 migration.

  """)


IOError


In [7]:
df

,Sno,Title,URL,Summary,Img_URL
0,1,Trump Moves to Replace Watchdog Who Identified...,https://www.nytimes.com/2020/05/01/us/politics...,"[This is a difficult question, because a lot d...",https://static01.nyt.com/images/2020/05/01/mul...
1,2,Coronavirus in the U.S.: Latest Map and Case C...,https://www.nytimes.com/interactive/2020/us/co...,None,https://static01.nyt.com/images/2020/03/03/us/...
2,3,Tracking the ‘Murder Hornet’: A Deadly Pest Ha...,https://www.nytimes.com/2020/05/02/us/asian-gi...,"[BLAINE, Wash. — In his decades of beekeeping,...",https://static01.nyt.com/images/2020/05/01/us/...
3,4,Coronavirus Live Updates: States Rely on Resid...,https://www.nytimes.com/2020/05/02/us/coronavi...,"[In places still under lockdown, the weekend i...",https://static01.nyt.com/images/2020/05/02/us/...
4,5,Coronavirus in N.Y.: Live Updates,https://www.nytimes.com/2020/05/02/nyregion/co...,"[Gov., Philip D. Murphy of New Jersey reported...",https://static01.nyt.com/images/2020/05/02/nyr...
5,6,What the Proponents of ‘Natural’ Herd Immunity...,https://www.nytimes.com/2020/05/01/opinion/cor...,[Some countries are attempting strategies inte...,https://static01.nyt.com/images/2020/05/03/opi...
6,7,How Long Will a Vaccine Really Take?,https://www.nytimes.com/interactive/2020/04/30...,[We’ve never released a coronavirus vaccine fo...,https://static01.nyt.com/images/2020/04/29/us/...
7,8,Investigate Tara Reade’s Allegations,https://www.nytimes.com/2020/05/01/opinion/bid...,"[Biden said that if such a document existed, t...",https://static01.nyt.com/images/2020/05/02/opi...
8,9,"Fearing Political Peril, Republicans Edge Away...",https://www.nytimes.com/2020/05/02/us/politics...,"[“It’s a tightrope,” he said of the challenge ...",https://static01.nyt.com/images/2020/04/30/us/...
9,10,Olympic Gymnast Recalls Emotional Abuse ‘So Tw...,https://www.nytimes.com/2020/05/01/sports/magg...,"[Coach Maggie Haney, left, watching Laurie Her...",https://static01.nyt.com/images/2020/05/02/spo...


In [8]:
t.close()

True

In [9]:
def insertNews(conn,news):    
    sql = ''' INSERT INTO news(site_id,create_date,title,url,summary,img_url,latest)
              VALUES(?,?,?,?,?,?,1) '''
    cur = conn.cursor()
    cur.execute(sql, news)
    return cur.lastrowid

def markNews(conn, site_id):
    cur = conn.cursor()
    cur.execute("UPDATE news set latest=0 where latest=1 and site_id=? ",(site_id,))

In [10]:
now = datetime.now()
# dd/mm/YY
dt_string = now.strftime("%d/%m/%Y")
markNews(conn, site_id)
for index, row in df.iterrows():
    title = row['Title'].decode('utf-8')
    url = row['URL']
    summary = str(row['Summary'])
    img_url = row['Img_URL']
    news = (site_id,dt_string,title,url,summary,img_url)
    news_id = insertNews(conn,news)
    print("News %d has been created" % news_id)
    
conn.commit()

News 64 has been created
News 65 has been created
News 66 has been created
News 67 has been created
News 68 has been created
News 69 has been created
News 70 has been created
News 71 has been created
News 72 has been created
News 73 has been created
News 74 has been created
News 75 has been created


In [11]:
conn.close()